<a href="https://colab.research.google.com/github/jakewalter/intro_seismology/blob/main/reflection_coefficient.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Reflection coefficients of SH waves

In [ ]:

import numpy as np
import matplotlib.pyplot as plt

def RTSH(p, mi, mt):
    etabi = csqrt(1/(mi[1]*mi[1]) - p**2)
    etabt = csqrt(1/(mt[1]*mt[1]) - p**2)
    mui = mi[1]**2 * mi[2]
    mut = mt[1]**2 * mt[2]

    Rss = (mui*etabi - mut*etabt) / (mui*etabi + mut*etabt)
    Tss = 2 * mui * etabi / (mui*etabi + mut*etabt)
    Tss[-1] = Tss[-1] * (-1j)

    RTmatrix = np.conj(np.vstack((Rss, Tss)).T)
    phase = np.unwrap(np.angle(RTmatrix)) * 180 / np.pi

    return RTmatrix, phase

mi = [3.9*1.732, 3.9, 2.9]
mt = [4.49*1.732, 4.49, 3.38]
vs = mi[1]

n = 100
p = np.linspace(0, 1/mi[1], n)

RTmatrix, phase = RTSH(p, mi, mt)
Rss = RTmatrix[:,0]
Tss = RTmatrix[:,1]
PRss = phase[:,0]
PTss = phase[:,1]

angle = (180/np.pi) * np.arcsin(p * vs)

ymin, ymax = -1.1, 2.2
xmin, xmax = 0, 90
ypmin, ypmax = -185, 185

# Plotting
plt.figure(figsize=(10, 8))

# Reflected SH: amplitude versus incident angle
plt.subplot(221)
plt.plot(angle, np.abs(Rss), 'k-', linewidth=2)
plt.plot(angle, np.real(Rss), 'k-', linewidth=0.5)
plt.plot(angle, np.imag(Rss), 'k:', linewidth=0.5)
plt.xlabel('Incidence Angle (°)')
plt.ylabel('Amplitude')
plt.ylim([ymin, ymax])
plt.xlim([xmin, xmax])
plt.title('Reflected wave')

# Transmitted SH: amplitude versus incident angle
plt.subplot(222)
plt.plot(angle, np.abs(Tss), 'k-', linewidth=2)
plt.plot(angle, np.real(Tss), 'k-', linewidth=0.5)
plt.plot(angle, np.imag(Tss), 'k:', linewidth=0.5)
plt.xlabel('Incidence Angle (°)')
plt.ylabel('Amplitude')
plt.ylim([ymin, ymax])
plt.xlim([xmin, xmax])
plt.title('Transmitted wave')

# Reflected SH: phase versus incident angle
plt.subplot(223)
plt.plot(angle, PRss, 'k-', linewidth=2)
plt.xlabel('Incidence Angle (°)')
plt.ylabel('Phase (°)')
plt.ylim([ypmin, ypmax])
plt.xlim([xmin, xmax])
plt.title('Reflected wave phase')

# Transmitted SH: phase versus incident angle
plt.subplot(224)
plt.plot(angle, PTss, 'k-', linewidth=2)
plt.xlabel('Incidence Angle (°)')
plt.ylabel('Phase (°)')
plt.ylim([ypmin, ypmax])
plt.xlim([xmin, xmax])
plt.title('Transmitted wave phase')

plt.tight_layout()
plt.savefig('figure64.pdf')
plt.show()

Reflection coefficients

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


from numpy.lib.scimath import sqrt as csqrt


def RTcoef(p, mi, mt):
    etaai = csqrt(1 / (mi[0] * mi[0]) - p * p)
    etaat = csqrt(1 / (mt[0] * mt[0]) - p * p)
    etabi = csqrt(1 / (mi[1] * mi[1]) - p * p)
    etabt = csqrt(1 / (mt[1] * mt[1]) - p * p)

    a = mt[2] * (1 - 2 * mt[1] * mt[1] * p * p) - mi[2] * (1 - 2 * mi[1] * mi[1] * p * p)
    b = mt[2] * (1 - 2 * mt[1] * mt[1] * p * p) + 2 * mi[2] * mi[1] * mi[1] * p * p
    c = mi[2] * (1 - 2 * mi[1] * mi[1] * p * p) + 2 * mt[2] * mt[1] * mt[1] * p * p
    d = 2 * (mt[2] * mt[1] * mt[1] - mi[2] * mi[1] * mi[1])

    E = b * etaai + c * etaat
    F = b * etabi + c * etabt
    G = a - d * etaai * etabt
    H = a - d * etaat * etabi
    D = E * F + G * H * p * p

    Rpp = ((b * etaai - c * etaat) * F - (a + d * etaai * etabt) * H * p * p) / D
    Rps = -(2 * etaai * (a * b + d * c * etaat * etabt) * p * mi[0] / mi[1]) / D
    Rss = -((b * etabi - c * etabt) * E - (a + d * etaat * etabi) * G * p * p) / D
    Rsp = -(2 * etabi * (a * b + d * c * etaat * etabt) * p * (mi[1] / mi[0])) / D
    Tpp = (2 * mi[2] * etaai * F * (mi[0] / mt[0])) / D
    Tps = (2 * mi[2] * etaai * H * p * (mi[0] / mt[1])) / D
    Tss = 2 * mi[2] * etabi * E * (mi[1] / mt[1]) / D
    Tsp = -2 * (mi[2] * etabi * G * p * (mi[1] / mt[0])) / D

    RTmatrix = np.array([Rpp, Rps, Rss, Rsp, Tpp, Tps, Tss, Tsp]).T
    phase = np.angle(RTmatrix) * 180 / np.pi

    return RTmatrix, phase

mi = [4.98, 2.9, 2.667]
mt = [8.00, 4.6, 3.38]

n = 200
p = np.linspace(0, 1/mi[0], n)

R, phase = RTcoef(p, mi, mt)
Rpp = R[:,0]
Rps = R[:,1]
Tpp = R[:,4]
Tps = R[:,5]

PRss = phase[:,2]
PTss = phase[:,6]

angle = (180/np.pi) * np.arcsin(p * mi[0])

ymin, ymax = -0.1, 2.0
xmin, xmax = 0, 90

plt.figure(figsize=(10, 8))

plt.subplot(221)
plt.plot(angle, np.abs(Rpp), 'k-')
#plt.xlabel('Incidence Angle (°)')
plt.ylabel('Amplitude')
plt.ylim([ymin, ymax])
plt.xlim([xmin, xmax])
plt.title('Reflected P')

plt.subplot(222)
plt.plot(angle, np.abs(Rps), 'k-')
#plt.xlabel('Incidence Angle (°)')
plt.ylabel('Amplitude')
plt.ylim([ymin, ymax])
plt.xlim([xmin, xmax])
plt.title('Reflected S')

plt.subplot(223)
plt.plot(angle, np.abs(Tpp), 'k-')
plt.xlabel('Incidence Angle (°)')
plt.ylabel('Amplitude')
plt.ylim([ymin, ymax])
plt.xlim([xmin, xmax])
plt.title('Transmitted P')

plt.subplot(224)
plt.plot(angle, np.abs(Tps), 'k-')
plt.xlabel('Incidence Angle (°)')
plt.ylabel('Amplitude')
plt.ylim([ymin, ymax])
plt.xlim([xmin, xmax])
plt.title('Transmitted S')

plt.tight_layout()
plt.savefig('sampleangle.pdf')
plt.show()

Attenuation and tstar

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Part 1: Time Scaling Without Frequency Modulation
plt.close('all')

f = np.arange(0, 1.01, 0.01)
ts = 1
A = np.exp(-2*np.pi*f*ts/2)

ts0 = np.arange(1, 11, 4)
colo = plt.get_cmap('viridis')(np.linspace(0, 1, len(ts0)))

for i, ts in enumerate(ts0):
    A = np.exp(-2*np.pi*f*ts/2)
    plt.figure(1)
    plt.semilogy(f, A, color=colo[i])
    plt.gca().set_xlabel('frequency (Hz)')
    plt.gca().set_ylabel('A(f)')
    plt.gca().set_title('Time Scaling Without Frequency Modulation')
    plt.gca().set_ylim([1e-10, 1])
    plt.gca().set_xlim([0, 1])
    plt.gca().tick_params(axis='both', which='major', labelsize=15)
    nfx = np.where(f == 0.8)[0]
    plt.text(0.8, A[nfx], f't*={ts}', fontsize=15)

    plt.figure(2)
    Ainve = np.fft.ifftshift(np.fft.ifft(A))
    t = np.linspace(1, -1, len(f))
    plt.plot(t, np.abs(Ainve), color=colo[i])
    plt.gca().set_xlabel('time')
    plt.gca().set_ylabel('A(t)')
    plt.gca().set_title('Time Scaling Without Frequency Modulation')
    plt.gca().set_xlim([-1, 1])
    plt.gca().tick_params(axis='both', which='major', labelsize=15)

plt.figure(1)
plt.savefig('tstar-nofrequency.png')
plt.figure(2)
plt.savefig('tstar-operator-nof.png')

# Part 2: Time Scaling with Frequency Modulation
f = np.arange(0.01, 1.01, 0.01)
ts = 1

for i, ts in enumerate(ts0):
    A = np.exp(-2*np.pi*f*ts/2) * np.exp(-1j*2*np.pi*ts*f*np.log(f/10)/np.pi) * np.exp(-1j*2*np.pi*f*1)

    plt.figure(3)
    plt.semilogy(f, np.abs(A), linewidth=2, color=colo[i])
    plt.semilogy(f, np.abs(np.real(A)), linewidth=0.5, color=colo[i])
    plt.semilogy(f, np.abs(np.imag(A)), ':', linewidth=0.5, color=colo[i])
    plt.gca().set_xlabel('frequency (Hz)')
    plt.gca().set_ylabel('A(f)')
    plt.gca().set_title('Time Scaling with Frequency Modulation')
    plt.gca().set_ylim([1e-10, 1])
    plt.gca().set_xlim([0, 1])
    plt.gca().tick_params(axis='both', which='major', labelsize=15)
    nfx = np.where(f == 0.8)[0]
    plt.text(0.8, np.abs(A[nfx]), f't*={ts}', fontsize=15)

    plt.figure(4)
    Ainve = np.fft.ifftshift(np.fft.ifft(A))
    t = np.linspace(1, -1, len(f))
    plt.plot(t, np.abs(Ainve), color=colo[i])
    plt.gca().set_xlabel('time')
    plt.gca().set_ylabel('A(t)')
    plt.gca().set_title('Time Scaling with Frequency Modulation')
    plt.gca().set_xlim([-1, 1])
    plt.gca().tick_params(axis='both', which='major', labelsize=15)

plt.figure(3)
plt.savefig('tstar-withf.png')
plt.figure(4)
plt.savefig('tstar-operator-withf.png')

plt.show()